In [1]:
# 統計等のライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import poisson
from scipy.stats import binom
import math

# ディレクトリ用のライブラリ
import os

# 可視化ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/100_業務関連/3001_監査法人verita/斎久工業/202303期末監査/サンプリング

/content/drive/.shortcut-targets-by-id/143vrgQuxKXXL4INCG0ZHJ7_SMUOSo3ev/サンプリング


In [63]:
# 読み込み用のシート名(.xlsxまで入れる)
file_name = 'サンプリング（中間監査） 2.xlsx'
sheet_name = '期日未収'
amount = '金額'

sample_data = pd.read_excel(file_name, sheet_name=sheet_name, header=1)

# 金額がマイナスなので、それを修正
sample_data[amount] = sample_data[amount]*-1

In [64]:
sample_data.head()

,本支店コード,伝票日付,伝票番号,伝票摘要名,部門コード,科目コード,科目名,補助科目１,金額,消費税,...,伝票摘要３,工事コード１,工事コード２,工事コード３,工事名称,完成予定日,取引先・社員コード,業者名称,支払先コード,支払先名称
0,10,20220428,400005,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,235840000,0,...,NaN,AA,201901D01,0,同愛記念病院整備事業計画（新棟建設）,20220630,31621.0,大成建設株式会社,NaN,NaN
1,10,20220711,400002,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,182470581,0,...,NaN,AA,201605S10,0,渋谷駅桜丘口地区第一種市街地再開発事業(A街区),20231130,30811.0,鹿島建設株式会社,NaN,NaN
2,15,20220510,400003,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,164944986,0,...,NaN,FF,202006S01,0,SAKURA PROJECT（先行外構ⅠGMP),20221101,30811.0,鹿島建設株式会社,NaN,NaN
3,14,20220608,400001,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,154000000,0,...,NaN,EE,202006Y01,0,米沢市立病院新病院棟建設工事,20231031,30515.0,株式会社フジタ,NaN,NaN
4,10,20220801,400001,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,132066000,0,...,NaN,AA,201901D01,0,同愛記念病院整備事業計画（新棟建設）,20220630,31621.0,大成建設株式会社,NaN,NaN


In [65]:
# 母集団の金額が正しいかチェック
total_amount = sample_data[amount].sum()
print(total_amount)

2847086751


In [66]:
# ポアソン分布による金額単位サンプリングによるサンプル数算定の関数
def sample_poisson(N, pm, ke, alpha, audit_risk, internal_control='依拠しない'):
    k = np.arange(ke+1)
    pt = pm/N
    n = 1
    while True:
        mu = n*pt
        pmf_poi = poisson.cdf(k, mu)
        if pmf_poi.sum() < alpha:
            break
        n += 1
    if audit_risk == 'SR':
        n = math.ceil(n)
    if audit_risk == 'RMM-L':
        n = math.ceil(n/10*2)
    if audit_risk == 'RMM-H':
        n = math.ceil(n/2)
    if internal_control == '依拠する':
        n = math.ceil(n/3)
    return n

In [67]:
# 変動パラメータの設定

# 母集団の金額合計
N =  total_amount
# 手続実施上の重要性
pm = 758665000
# ランダムシード　(サンプリングの並び替えのステータスに利用、任意の数を入力)
random_state = 0
# 監査リスク
audit_risk = 'RMM-L'
# 内部統制
internal_control = '依拠しない'


# 予想虚偽表示金額（変更不要）
ke = 0
alpha = 0.05

# サンプルサイズnの算定
n = sample_poisson(N, pm, ke, alpha, audit_risk, internal_control)
print(n)

# サンプリングシートに記載用の、パラメータ一覧
sampling_param = pd.DataFrame([['母集団合計', N],
                               ['手続実施上の重要性', pm],
                               ['リスク', audit_risk],
                               ['内部統制', internal_control],
                               ['random_state', random_state]])

3


In [68]:
# 母集団をまずは降順に並び替える（ここで並び替えるのは、サンプル出力の安定のため安定のため）
sample_data = sample_data.sort_values(amount, ascending=False)

# 母集団をシャッフル
shuffle_data = sample_data.sample(frac=1, random_state=random_state) #random_stateを使って乱数を固定化する
shuffle_data.head()

,本支店コード,伝票日付,伝票番号,伝票摘要名,部門コード,科目コード,科目名,補助科目１,金額,消費税,...,伝票摘要３,工事コード１,工事コード２,工事コード３,工事名称,完成予定日,取引先・社員コード,業者名称,支払先コード,支払先名称
510,15,20220525,400004,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,77000,0,...,NaN,FF,200106D01,581,大日本印刷㈱柏工場DNPE各所雑改修工事(72-4),20220228,20531.0,株式会社ＤＮＰエンジニアリング,NaN,NaN
457,32,20220608,400001,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,126500,0,...,NaN,WW,202111D02,0,大日本印刷㈱戸畑工場2期棟西プラット横マンホール取替工事,20211210,20540.0,大日本印刷株式会社,NaN,NaN
440,15,20220825,400003,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,140800,0,...,NaN,FF,201202D01,66,大日本印刷㈱牛久工場DNPE各所雑改修工事(72-6),20220331,20531.0,株式会社ＤＮＰエンジニアリング,NaN,NaN
296,10,20220708,400001,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,209000,0,...,NaN,AA,202104D03,13,大日本印刷上福岡工場３月迄諸口工事,20220331,20540.0,大日本印刷株式会社,NaN,NaN
37,10,20220428,400005,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,13893000,0,...,NaN,AA,201810D02,0,大日本印刷市谷工場整備計画建設工事(C工区),20221130,31621.0,大成建設株式会社,NaN,NaN


In [69]:
# サンプリング区間の算定
m = N/n
print(m)

949028917.0


In [70]:
# 列の追加
shuffle_data['cumsum'] = shuffle_data[amount].cumsum() # 積み上げ合計
shuffle_data['group'] = shuffle_data['cumsum']//m # サンプルのグループ化
shuffle_data.head()

result_data = shuffle_data.loc[shuffle_data.groupby('group')['cumsum'].idxmin(), ]
result_data

,本支店コード,伝票日付,伝票番号,伝票摘要名,部門コード,科目コード,科目名,補助科目１,金額,消費税,...,工事コード２,工事コード３,工事名称,完成予定日,取引先・社員コード,業者名称,支払先コード,支払先名称,cumsum,group
510,15,20220525,400004,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,77000,0,...,200106D01,581,大日本印刷㈱柏工場DNPE各所雑改修工事(72-4),20220228,20531.0,株式会社ＤＮＰエンジニアリング,NaN,NaN,77000,0.0
6,14,20220930,800015,ファクタリング科目振替自動仕訳,NaN,1520,期日支払未収入金,NaN,104720000,0,...,202012N02,0,ニプロ大舘第7工場新築工事（PES棟）,20221220,NaN,NaN,NaN,NaN,952575727,1.0
13,10,20220610,400002,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,67936883,0,...,202012H01,0,HANEDA INNOVATIONCITY増築工事,20230630,30811.0,鹿島建設株式会社,NaN,NaN,1925646739,2.0
559,10,20220425,400004,ファクタリング決済自動仕訳,NaN,1520,期日支払未収入金,NaN,21780,0,...,202104D03,14,大日本印刷㈱鶴瀬工場12月迄諸口工事(DNPE),20220131,20531.0,株式会社ＤＮＰエンジニアリング,NaN,NaN,2847086751,3.0


In [71]:
file_name = '{}サンプル.xlsx'.format(sheet_name)
# result_data.to_excel(file_name, encoding="shift_jis", index=False)

In [72]:
writer = pd.ExcelWriter(file_name)

# 全レコードを'全体'シートに出力
sample_data.to_excel(writer, sheet_name = '母集団', index=False)
# サンプリング結果を、サンプリングシートに記載
result_data.to_excel(writer, sheet_name = 'サンプリング結果', index=False)
# サンプリングの情報追記
sampling_param.to_excel(writer, sheet_name = 'サンプリングパラメータ', index=False, header=None)


# Excelファイルを保存
writer.save()
# Excelファイルを閉じる
writer.close()

In [3]:
pd.read_excel('C:/Users/rhamano/Downloads/sample.xlsx')

,本支店コード,伝票日付,伝票摘要名,部門コード,科目コード,金額
0,0,20221101,ファクタリング決済自動仕訳\t,NaN,1500,1000
1,1,20221102,ファクタリング決済自動仕訳\t,NaN,1501,2500
2,2,20221103,ファクタリング決済自動仕訳\t,NaN,1502,3000
3,3,20221104,ファクタリング決済自動仕訳\t,NaN,1503,5000
4,4,20221105,ファクタリング決済自動仕訳\t,NaN,1504,10000
5,5,20221106,ファクタリング決済自動仕訳\t,NaN,1505,54300
